In [59]:
'''Leaky integrated firing neuron model
Few feautures :
Spatial network, no memory term but exponential dacy of cumulative dose

This is a minor change of the Dodds and Watts model
Adaptive rewiring has still not been implemented'''

'Leaky integrated firing neuron model\nFew feautures :\nSpatial network, no memory term but exponential dacy of cumulative dose\n\nThis is a minor change of the Dodds and Watts model\nAdaptive rewiring has still not been implemented'

In [60]:
#importing the required libraries

import networkx as nx #network library to generate inital networks and perform various network operations
import matplotlib.pyplot as plt #for plotting
import numpy as np #for using arrays and vectorising the code wherever possible
import scipy
import random
import cProfile #for timing the loops
from numba import jit #numba precomplier to make the code faster
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import time

In [61]:
'''
variables used in the simulation
'''

N = 100
gamma = 1 #per second
T = 50 #total number of seconds
dt = 0.1 #seconds
d_star = 3
p = 0.1
r = 1
rho = 1
circ_rad = N/10
dose_quantity = 5

# nn = 2
# p_rew = 0.5

In [62]:
'''creating the network'''

'creating the network'

In [63]:
# #creating the static Watts-Strogatz network
# def static_network(N,nn,p_rew) :
#     G = nx.watts_strogatz_graph(N,nn,p_rew) #no rewiring for now

#     #obtaining the edges via the adjancency matrix
#     adj_mat = nx.adjacency_matrix(G,nodelist=range(N))
#     adj_mat = scipy.sparse.csr_matrix.toarray(adj_mat)
    
#     return G, adj_mat

In [64]:
'''
obtains the number of nodes N
generates N points
calculates euclidean distance between each pair of points
returns the coordinates of the points and the distance matrix which is N*N
'''
def calc_dist(N) :
    coords = []
    for counter in range(N) :
        coords.append((N*np.random.random(),N*np.random.random()))

    dist_mat = scipy.spatial.distance.cdist(coords,coords)
    return dist_mat,coords

In [65]:
'''network = connect to points withing a circle of radius'''
#creates a link between points/nodes which satisfies the conditions of the network
def coupling(dist_mat) :
    edge_list = []
    edge_mask = (dist_mat<circ_rad) & (dist_mat>0)
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(edge_mask))))
#     print(type(np.ndarray.tolist(np.transpose(np.where(edge_mask)))))
    return edge_list

In [66]:
dist_mat,coords = calc_dist(N)
edge_list = coupling(dist_mat)

In [67]:
'''
plot of how the network looks like
'''

# %matplotlib qt
plt.plot([coords[i][0] for i in range(len(coords))],[coords[i][1] for i in range(len(coords))],marker='o',markersize=12,linestyle='None')
i = 0
for counter in range(len(edge_list)):
    plt.plot([coords[edge_list[i][0]][0],coords[edge_list[i][1]][0]],\
             [coords[edge_list[i][0]][1],coords[edge_list[i][1]][1]],'black',alpha=0.1)
    i = i + 1

In [68]:
adj_mat = np.int64(np.zeros((N,N)))
for i in range(len(edge_list)):
    adj_mat[edge_list[i][0],edge_list[i][1]] = np.int64(1)

In [69]:
'''
network has been created
1. distance matrix
2. edge_list 
3. adjacency matrix '''

'\nnetwork has been created\n1. distance matrix\n2. edge_list \n3. adjacency matrix '

In [70]:
def dose(adj_mat,p,partner_state_prev,d) :

    z1 = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    dose_transfer_mask = p > z1
    mod_adj_mat = np.multiply(adj_mat,z1) #modifying adjacency matrix to include the proability of dose transfer
#     print(mod_adj_mat)
    dose_mask = (p>mod_adj_mat) & (adj_mat!=0) & (partner_state_prev == 2)
#     print(dose_mask)
    d[dose_mask] = dose_quantity #whenever it is proabable, dose transfer occurs
    
    return d

In [71]:
def cumu_dose(d,D_prev,gamma) :
    I = d.sum(axis=1).reshape(N,1)
#     I = 0
    D = (D_prev - (gamma*D_prev*dt)) + I
#     print(I)
    return D

In [72]:
def upd_indi_state(D,d_star,indi_state_prev) :
    
    z2 = np.random.uniform(size=(N,1))
    z3 = np.random.uniform(size=(N,1))
    
    indi_state = indi_state_prev
    
    indi_state_mask1 = (D>=d_star) & (indi_state_prev==1)
    indi_state[indi_state_mask1] = 2
    
    indi_state_mask21 = (D<d_star) & (indi_state_prev==2) & (r>=z2) & (rho>=z3)
    indi_state[indi_state_mask21] = 1
    
    indi_state_mask22 = (D<d_star) & (indi_state_prev==2) & (r>=z2) & (rho<z3)
    indi_state[indi_state_mask22] = 3
    
    indi_state_mask23 = (D<d_star) & (indi_state_prev==2) & (r<z2)
    indi_state[indi_state_mask23] = 2
    
    return indi_state

In [73]:
def states(indi_state) :
    infec_indi = []
    suscep_indi = []
    infec_indi = np.transpose(np.where(indi_state==2))
    suscep_indi = np.transpose(np.where(indi_state==1))
    return infec_indi,suscep_indi

In [74]:
#opening a file to store data
f = open("blah.txt","w+") 

In [75]:
t = np.arange(0,T,dt)
timeseries = pd.DataFrame(index=range(len(t)+1),columns=range(N))
# timeseries

In [76]:
%matplotlib qt
# G, adj_mat = static_network(N,nn,p_rew)

indi_state = np.random.randint(1,3,size=(N,1))
partner_state = np.random.randint(1,3,size=(N,1))
# print(indi_state)

indi_state_prev = indi_state
partner_state_prev = partner_state
# print('indi state =', np.shape(np.transpose(indi_state)))
# print('partner state =', partner_state)

d = 999*np.zeros((len(adj_mat),len(adj_mat[0])))

D = np.zeros((N,1))
D[np.where(indi_state==2)] = 4
D_prev = D
# print(D_prev)
timeseries.iloc[0,:] = np.transpose(indi_state)
dataframe_index_1 = 1

'''including the initial state in the plot'''

infec_indi = np.transpose(np.where(indi_state==2))
i = 0
for counter in range(len(edge_list)):
    plt.plot([coords[edge_list[i][0]][0],coords[edge_list[i][1]][0]],\
             [coords[edge_list[i][0]][1],coords[edge_list[i][1]][1]],'black',alpha=0.1)
    i = i + 1
plt.plot([coords[i][0] for i in range(len(coords))],[coords[i][1] for i in range(len(coords))],'blue',marker='o',markersize=12,linestyle='None')
plt.plot([coords[infec_indi[i][0]][0] for i in range(len(infec_indi))],[coords[infec_indi[i][0]][1] for i in range(len(infec_indi))],'red',marker='o',markersize=12,linestyle='None')

plt.title(t)
plt.pause(0.1)

'''the main part of the simulation'''
for t in np.arange(dt,T,dt) :
#     print(t)
    infec_indi = []
    d = 999*np.zeros((len(adj_mat),len(adj_mat[0])))
    d = dose(adj_mat,p,partner_state_prev,d)
    D = cumu_dose(d,D_prev,gamma)
    indi_state = upd_indi_state(D,d_star,indi_state_prev)
    
    infec_indi, suscep_indi = states(indi_state)
    
    plt.plot([coords[suscep_indi[i][0]][0] for i in range(len(suscep_indi))],[coords[suscep_indi[i][0]][1] for i in range(len(suscep_indi))],'blue',marker='o',markersize=12,linestyle='None')
    plt.plot([coords[infec_indi[i][0]][0] for i in range(len(infec_indi))],[coords[infec_indi[i][0]][1] for i in range(len(infec_indi))],'red',marker='o',markersize=12,linestyle='None')
    plt.title(t)
    plt.pause(0.1)
#     time.sleep(0.1)
    
    partner_state = indi_state
    indi_state_prev = indi_state
    partner_state_prev = partner_state
    D_prev = D
#     f.write("%f\t%f\n" % (t,D[0,0]))
    D = np.zeros((N,1))
#     timeseries.iloc[dataframe_index_1,:] = np.transpose(indi_state)
#     dataframe_index_1 = dataframe_index_1 + 1

KeyboardInterrupt: 

In [77]:
t = np.arange(0,T,dt)
# timeseries
# timeseries.plot()

In [78]:
blah = []
len(blah)

0